<div style="font-size: 14;color:red;font-size:25PX;font-family: arial;" align ="rigth" dir="rtl">
<b>
1- فراخوان توابع مورد نیاز
</b>
</div>

In [3]:
import os
import numpy as np
import pandas as pd
import cv2 
#working_dir = os.getcwd()
working_dir ='..'  # Use on Jupyter Notebook

<div style="font-size: 14;color:red;font-size:25PX;font-family: arial;" align ="rigth" dir="rtl">
<b>
2- خواندن تصویر و تبدیل به سیاه و سفید
</b>
</div>

In [4]:
img = cv2.imread(f"{working_dir}/opencv-images/isotermal-b3-1_5.jpg", cv2.IMREAD_GRAYSCALE)
(thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

<div style="font-size: 14;color:red;font-size:25PX;font-family: arial;" align ="rigth" dir="rtl">
<b>
3- تبدیل اطلاعات تصویر به دیتافریم
</b>
</div>

In [ ]:
dic = {}
for i in range(522):
    dic[i] = im_bw[:,i]
df = pd.DataFrame(dic)
#----------------- Remove the first 7 rows and the last 6 rows -----------------
df.drop(index=[0,1,2,3,4,5,6,329,330,331,332,333,334],inplace=True)
#----------------- Remove the first 12 columns and the last 12 columns -----------------
df.drop(columns=[0,1,2,3,4,5,6,7,8,9,10,11,514,515,516,517,518,519,520,521],inplace=True)
df.reset_index(drop=True,inplace=True)
df.columns = range(df.shape[1])
print(df.shape)
df.to_csv(f"{working_dir}/opencv-images/1.csv", index=True,header=True)
del df

(322, 502)


<div style="font-size: 14;color:red;font-size:25PX;font-family: arial;" align ="rigth" dir="rtl">
<b>
4- تبدیل اطلاعات تصویر بصورت x و y و label
</b>
</div>

In [ ]:
df = pd.read_csv(f"{working_dir}/opencv-images/1.csv",index_col=0)
print(df.shape)

dic_xscale =dict(zip(range(0,df.shape[1]),np.linspace(start=55,stop=0,num=df.shape[1],endpoint=True).round(2)))
dic_yscale =dict(zip(range(0,df.shape[0]),np.linspace(start=0,stop=18,num=df.shape[0],endpoint=True).round(2)))
dicxy = {}
xlist = []
ylist = []
label = []
for i in range(df.shape[1]):
    for j in range(df.shape[0]):
        if df.iloc[j,i]== 255:
            xlist.append(dic_xscale[i])
            ylist.append(dic_yscale[j])
            label.append(1)
        if df.iloc[j,i]== 0:
            xlist.append(dic_xscale[i])
            ylist.append(dic_yscale[j])
            label.append(0)
dicxy['x'] = xlist
dicxy['y'] = ylist
dicxy['label'] = label
xydf = pd.DataFrame(dicxy)




#xydf.to_csv(f"{working_dir}/opencv-images/1xy.csv", index=True,header=True)
del xydf,df,dicxy

(322, 502)
